In [1]:
import pandas as pd
import numpy as np

In [19]:
# load data

df = pd.read_csv('data/athlete_profile_dataset.csv', index_col='Unnamed: 0')
df

,Date,Time,Nom du fichier,Durée de déplacement,Distance,Fréquence cardiaque moyenne,Fréquence cardiaque maximum,Vitesse moyenne,Cadence moyenne,Puissance moyenne,...,new_relative_effort,TSS,HRR,trimp,HRSS,Fitness,Fitness Diff,Fatigue,Fatigue Diff,Form
0,2014-04-15,14:35:30,activities/142879521.tcx.gz,112.52,47.67,127.316002,176.0,NaN,73.387299,219.221359,...,63.0,NaN,0.498528,93.0,55.0,1.3,1.3,7.3,7.3,NaN
1,2014-04-17,14:29:30,activities/143568422.tcx.gz,73.92,33.36,130.358002,182.0,NaN,79.790298,241.315857,...,83.0,NaN,0.522864,68.0,40.0,2.2,0.9,10.8,4.5,-5.0
2,2014-04-19,10:56:24,activities/144313052.tcx.gz,217.67,85.62,139.462006,183.0,NaN,72.434898,234.804291,...,218.0,NaN,0.595696,260.0,153.0,5.7,3.5,28.5,19.1,-7.3
3,2014-04-21,14:27:58,activities/145168490.tcx.gz,50.32,25.62,111.242996,144.0,NaN,96.695900,184.828018,...,11.0,NaN,0.369944,24.0,14.0,5.7,0.2,23.3,-1.4,-19.2
4,2014-04-22,12:44:13,activities/145556830.tcx.gz,202.33,88.02,143.753006,174.0,NaN,76.811096,240.926056,...,172.0,NaN,0.630024,273.0,161.0,9.4,3.7,41.6,18.3,-17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1172,2023-02-15,09:29:58,activities/9194335010.fit.gz,25.45,18.11,155.882782,NaN,42.70,94.063995,375.270477,...,34.0,47.0,0.727062,48.0,28.0,33.5,-0.7,53.9,-7.5,-27.2
1173,2023-02-16,10:59:50,activities/9201718878.fit.gz,218.73,139.03,147.699341,NaN,38.14,83.431664,290.123718,...,220.0,281.0,0.661595,330.0,194.0,37.3,3.8,72.6,18.6,-20.4
1174,2023-02-16,10:31:27,activities/9201719852.fit.gz,8.97,4.04,107.026070,NaN,27.03,69.140045,168.564240,...,1.0,4.0,0.336209,4.0,2.0,37.3,3.8,72.6,18.6,-20.4
1175,2023-02-17,09:39:32,activities/9206565665.fit.gz,143.42,73.94,105.816284,NaN,30.93,75.174011,185.966919,...,19.0,86.0,0.326530,56.0,33.0,37.2,-0.1,67.3,-5.3,-35.3


In [20]:
df.isna().sum()

Date                             0
Time                             0
Nom du fichier                   0
Durée de déplacement             0
Distance                         0
Fréquence cardiaque moyenne      0
Fréquence cardiaque maximum    344
Vitesse moyenne                652
Cadence moyenne                124
Puissance moyenne                3
Poids de l'athlète             341
Puissance moyenne pondérée     206
intensity_score                206
Mesure d'effort                 50
new_relative_effort              0
TSS                            206
HRR                              0
trimp                            0
HRSS                             0
Fitness                          0
Fitness Diff                     0
Fatigue                          0
Fatigue Diff                     0
Form                             1
dtype: int64

## First Model

In [21]:
df_model = df.copy()

df_model['Vitesse moyenne'] = df_model['Vitesse moyenne'].fillna(df_model['Vitesse moyenne'].mean())
df_model['Cadence moyenne'] = df_model['Cadence moyenne'].fillna(df_model['Cadence moyenne'].mean())
df_model['Puissance moyenne'] = df_model['Puissance moyenne'].fillna(df_model['Puissance moyenne'].mean())

df_model.dropna(subset=['Puissance moyenne pondérée', 'intensity_score', 'Mesure d\'effort'], inplace=True)

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error


# Split data into X and y
X = df_model[['Distance', 'Vitesse moyenne', 'Fréquence cardiaque moyenne', "Durée de déplacement", "Cadence moyenne", "Puissance moyenne pondérée", 'intensity_score',
       "Mesure d'effort", 'TSS', 'HRR', 'trimp', 'HRSS']]
y = df_model[['Fitness', 'Fatigue']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with the mean value along each column
imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse}")

Mean squared error: 90.39859767837838


In [23]:
print(y_pred, y_test)

[[38.705 34.002]
 [35.397 33.176]
 [32.9   36.573]
 [35.273 40.793]
 [35.33  34.485]
 [36.575 43.016]
 [38.19  40.692]
 [31.277 30.948]
 [32.778 30.292]
 [24.824 24.347]
 [29.624 27.047]
 [30.777 35.401]
 [34.095 32.051]
 [34.001 31.583]
 [33.358 28.108]
 [27.06  29.942]
 [43.811 61.887]
 [36.619 45.489]
 [37.584 45.738]
 [33.577 29.38 ]
 [37.397 35.865]
 [39.674 41.051]
 [36.69  53.833]
 [34.835 38.711]
 [35.32  40.082]
 [33.005 30.46 ]
 [19.1   37.058]
 [32.572 31.012]
 [39.486 57.398]
 [34.2   31.273]
 [39.604 43.079]
 [39.192 50.286]
 [32.149 33.316]
 [39.243 44.423]
 [37.685 40.808]
 [26.119 30.116]
 [29.839 31.754]
 [27.67  36.727]
 [30.783 28.867]
 [32.843 32.748]
 [34.375 41.608]
 [32.545 28.409]
 [31.445 31.65 ]
 [33.049 40.025]
 [31.805 37.939]
 [29.496 27.403]
 [29.714 35.42 ]
 [30.963 30.994]
 [36.478 41.495]
 [25.096 29.583]
 [31.592 29.986]
 [34.682 30.32 ]
 [40.25  49.117]
 [30.628 32.184]
 [32.193 28.608]
 [30.857 31.693]
 [35.499 41.133]
 [19.282 24.373]
 [41.392 59.33

In [37]:
from tpot import TPOTRegressor
from tqdm import tqdm

# Split data into X and y
X = df_model[['Distance', 'Fréquence cardiaque moyenne', "Durée de déplacement", "Cadence moyenne", "Puissance moyenne pondérée", 'intensity_score',
       "Mesure d'effort", 'TSS', 'HRR', 'trimp', 'HRSS', 'Fitness', 'Fatigue']]
y = df_model['Form']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize TPOT regressor
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)

# Fit TPOT regressor
with tqdm(total=tpot.generations) as pbar:
    for i in range(tpot.generations):
        tpot.fit(X_train, y_train)
        tpot.evaluated_individuals_
        tpot.export('tpot_pipeline.py')
        pbar.update(1)

# Score TPOT regressor
mse = mean_squared_error(y_test, tpot.predict(X_test))
print('MSE:', mse)

# Print best pipeline
print(tpot.fitted_pipeline_)


  0%|          | 0/5 [00:00<?, ?it/s]

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

  0%|          | 0/5 [00:29<?, ?it/s]


Generation 1 - Current best internal CV score: -6.328491763474665


  0%|          | 0/5 [00:49<?, ?it/s]

...

 80%|████████  | 4/5 [08:52<01:46, 106.09s/it]


Generation 5 - Current best internal CV score: -5.909371202238718

Best pipeline: ElasticNetCV(PolynomialFeatures(ElasticNetCV(input_matrix, l1_ratio=0.45, tol=0.0001), degree=2, include_bias=False, interaction_only=False), l1_ratio=0.15000000000000002, tol=0.1)


100%|██████████| 5/5 [08:53<00:00, 106.64s/it]

MSE: 5.101361726069388
Pipeline(steps=[('stackingestimator',
                 StackingEstimator(estimator=ElasticNetCV(l1_ratio=0.45))),
                ('polynomialfeatures', PolynomialFeatures(include_bias=False)),
                ('elasticnetcv',
                 ElasticNetCV(l1_ratio=0.15000000000000002, tol=0.1))])



C:\Users\chach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but ElasticNetCV was fitted with feature names
  warnings.warn(


In [28]:
# Print best pipeline
print(tpot.fitted_pipeline_)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(include_bias=False)),
                ('elasticnetcv', ElasticNetCV(l1_ratio=0.9, tol=0.01))])


In [29]:
# Export the best pipeline as Python code
tpot.export('tpot_pipeline.py')

In [33]:
import dill
# Save the TPOT pipeline as a file
with open('tpot_pipeline.pkl', 'wb') as f:
    dill.dump(tpot.fitted_pipeline_, f)

In [34]:
# Load the saved TPOT pipeline from file
with open('tpot_pipeline.pkl', 'rb') as f:
    tpot_pipeline = dill.load(f)

def predict_fitness_score(distance, av_hr, duration):
    # Calculate additional features
    speed = distance / duration
    power = 2.8 * speed + 0.0098 * (speed ** 3)
    
    # Create input DataFrame
    input_data = pd.DataFrame({
        'Distance': [distance],
        'Fréquence cardiaque moyenne': [av_hr],
        'Durée de déplacement': [duration],
        'Vitesse moyenne': [speed],
        'Puissance moyenne': [power]
    })
    
    # Make prediction using TPOT pipeline
    predicted_fitness = tpot_pipeline.predict(input_data)
    
    return predicted_fitness[0]


In [35]:
predict_fitness_score(140, 180, 120)

C:\Users\chach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Puissance moyenne
Feature names seen at fit time, yet now missing:
- Cadence moyenne
- Fatigue
- Fitness
- HRR
- HRSS
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 5 features, but PolynomialFeatures is expecting 14 features as input.